In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd

df = pd.read_csv('health_result.csv')
df.head()

,question,answer,category02
0,"너무 목이 마른데요, 혹시 정수기 같은 건 없나요?",정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.,29.대기실 및 진료실 위치 안내
1,교정기가 떨어진 것 같아서 붙이러 왔어요.,접수 먼저 부탁드립니다. 카운터에서 성함을 입력해 주시길 바랍니다.,27.진료 접수 안내
2,앞으로 얼마나 나아지는지 보고 아프면 오라고 하네요. 제가 다음에 예약하고 와도 되죠?,예약하실 때 지난 번 진료 기록이 있다고 말씀해 주시길 부탁 드립니다. 감사합니다.,33.다음 진료실 예약
3,이것도 수납용 키오스크인가요? 대기 줄이 너무 길어서요.,이곳에서도 수납 진행 가능합니다. 홈버튼을 누른 후 수납 버튼을 누르십시오.,31.수납 방법 안내
4,주사실 안에 가서 있으라는데 주사실이 어딘데요?,침을 맞는 곳이라 이름이 주사실입니다. 3층으로 올라가시면 됩니다.,29.대기실 및 진료실 위치 안내


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # KcBERT 모델 로드 (한국어 모욕성 표현 감지)
# tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
# model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base")

# def detect_offensive(sentence):
#     # 디바이스 설정 (GPU 사용 가능 여부 확인)
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # 모델을 GPU로 이동
#     model.to(device)
#     inputs = tokenizer(sentence, return_tensors="pt").to(device)
#     outputs = model(**inputs)
#     prediction = outputs.logits.argmax(dim=-1)
#     return prediction.item()  # 0: 정상 문장, 1: 모욕적 문장

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModel.from_pretrained("beomi/KcBERT-base")

# 질문 리스트
questions = df['question'].tolist()

# 질문들의 임베딩 벡터 미리 계산
def compute_question_embeddings(questions):
    embeddings = []
    for question in questions:
        inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # 문장 전체의 임베딩을 평균으로 취함
        embedding = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(embedding)
    return embeddings

# 질문에 대한 임베딩 계산
question_embeddings = compute_question_embeddings(questions)

# 임베딩을 저장할 때 torch.save() 사용 가능
torch.save(question_embeddings, 'question_embeddings.pth')


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\tlsek\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tlsek\.cache\huggingface\hub\models--beomi--KcBERT-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [4]:
import torch.nn.functional as F

question_embeddings = torch.load('question_embeddings.pth')

# 입력 문장을 임베딩으로 변환
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 임베딩 벡터 반환

# 입력 질문과 사전 임베딩 간의 코사인 유사도 계산
def find_most_similar_answer(input_question, question_embeddings, answers):
    input_embedding = get_embedding(input_question)  # 입력 질문 임베딩

    max_similarity = -1
    best_answer = None

    # 각 사전 계산된 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        similarity = F.cosine_similarity(input_embedding, question_embedding).item()

        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer

# 입력 질문 예시
input_question = "정수기 위치가 어디죠?"
answers = df['answer'].tolist()

# 유사도 기반 답변 찾기
answer = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)


C:\Users\tlsek\AppData\Local\Temp\ipykernel_18512\1478106187.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('question_embeddings.pth')


챗봇 답변: 위치 정보를 조회하겠습니다. 병원에서 50m 떨어져 있는 곳에 위치해 있습니다.


In [5]:
# 질문을 토크나이즈
encodings = tokenizer(questions, truncation=True, padding=True, return_tensors="pt")

# 대답을 레이블로 변환
labels = torch.tensor([answers.index(a) for a in answers])

label_to_idx = {label: idx for idx, label in enumerate(set(answers))}
labels = torch.tensor([label_to_idx[a] for a in answers])

In [6]:
# Dataset 클래스 정의
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Dataset 객체 생성
dataset = QADataset(encodings, labels)

In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

In [ ]:
# 모델 학습
trainer.train()

Step,Training Loss
10,9.192900
20,9.161100
30,9.122700
40,9.160000
50,9.128800
60,9.124600
70,9.213800
80,9.104600
90,9.172400
100,9.109700


TrainOutput(global_step=6459, training_loss=5.941141075681174, metrics={'train_runtime': 1106.4168, 'train_samples_per_second': 93.402, 'train_steps_per_second': 5.838, 'total_flos': 2294301724157520.0, 'train_loss': 5.941141075681174, 'epoch': 3.0})

In [ ]:
import torch
def get_response(question):
    # 디바이스 설정 (GPU 사용 가능 여부 확인)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 모델을 GPU로 이동
    model.to(device)

    # 입력 데이터를 GPU로 이동
    inputs = tokenizer(question, return_tensors="pt").to(device)

    # 출력도 GPU에서 실행
    outputs = model(**inputs)

    # 가장 높은 점수를 받은 레이블을 예측
    predicted_label = outputs.logits.argmax(dim=-1).item()

    # 예측된 레이블을 실제 답변으로 변환
    response = answers[predicted_label]

    return response

# 예시 질문
question = "너무 목이 마른데요, 혹시 정수기 같은 건 없나요?"
response = get_response(question)
print(response)  # 출력: "정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다."

예약을 원하시는 날짜 및 요일을 정확히 말씀해 주시기 바랍니다.


In [ ]:
def chatbot_response(question):
    # 모욕성 발언 감지
    is_offensive = detect_offensive(question)

    if is_offensive == 1:
        return "모욕적인 발언이 감지되었습니다. 다른 질문을 해주세요."
    else:
        # 모욕적이지 않으면, 컨텍스트에서 답변 추출
        response = get_response(question)
        return response

In [ ]:
# 예시 질문
question = "이 씨발아 너무 목이 마른데?"
response = chatbot_response(question)
answer = get_response(question)
print(answer)
print(response)  # 출력: "정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다."

접수 먼저 부탁드립니다. 카운터에서 성함을 입력해 주시길 바랍니다.
모욕적인 발언이 감지되었습니다. 다른 질문을 해주세요.


In [ ]:
# 예시 질문
question = "화장실은 어딨어?"
response = chatbot_response(question)
answer = get_response(question)
print(answer)
print(response)  # 출력: "정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다."

접수 먼저 부탁드립니다. 카운터에서 성함을 입력해 주시길 바랍니다.
모욕적인 발언이 감지되었습니다. 다른 질문을 해주세요.


In [ ]:
# 모델 저장
trainer.save_model('./saved_model')  # 저장할 디렉토리 지정

# 토크나이저도 함께 저장
tokenizer.save_pretrained('./saved_model')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 저장된 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained('./saved_model')
tokenizer = AutoTokenizer.from_pretrained('./saved_model')